In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

df = pd.read_csv('cansim-0800020-eng-6674700030567901031.csv', skiprows=6, skipfooter=9, engine='python')

In [2]:
from pandas.tseries.offsets import MonthEnd
df['Adjustments'] = pd.to_datetime(df['Adjustments']) + MonthEnd(1)

In [3]:
df = df.set_index('Adjustments')

In [4]:
split_date = pd.Timestamp(2011,1,1)

In [5]:
train = df.loc[:split_date,['Unadjusted']]
test = df.loc[split_date:,['Unadjusted']]

In [6]:
from sklearn.preprocessing import MinMaxScaler

In [7]:
sc = MinMaxScaler()

In [8]:
train_sc = sc.fit_transform(train)
test_sc = sc.transform(test)

In [9]:
train_sc_df = pd.DataFrame(train_sc, columns=['Scaled'], index=train.index)
test_sc_df = pd.DataFrame(test_sc, columns=['Scaled'], index=test.index)

In [10]:
for s in range(1,13):
    train_sc_df['shift_{}'.format(s)] = train_sc_df['Scaled'].shift(s)
    test_sc_df['shift_{}'.format(s)] = test_sc_df['Scaled'].shift(s)

In [11]:
x_train = train_sc_df.dropna().drop('Scaled', axis=1)
y_train = train_sc_df.dropna()[['Scaled']]

In [12]:
x_test = test_sc_df.dropna().drop('Scaled', axis=1)
y_test = test_sc_df.dropna()[['Scaled']]

In [13]:
x_train = x_train.values
x_test = x_test.values

y_train = y_train.values
y_test = y_test.values

In [14]:
x_train_t = x_train.reshape(x_train.shape[0],12,1)
x_test_t = x_test.reshape(x_test.shape[0],12,1)

In [15]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.callbacks import EarlyStopping

Using TensorFlow backend.
C:\Users\iks15\anaconda3\envs\mltest\lib\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\iks15\anaconda3\envs\mltest\lib\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\iks15\anaconda3\envs\mltest\lib\site-packages\tensorflow\python\framework\dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\iks15\anaconda3\envs\mltest\lib\sit

In [16]:
K.clear_session()

In [17]:
model = Sequential()

In [18]:
model.add(LSTM(20, input_shape=(12, 1)))

In [19]:
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20)                1760      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 21        
Total params: 1,781
Trainable params: 1,781
Non-trainable params: 0
_________________________________________________________________


In [20]:
early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)

model.fit(x_train_t, y_train, epochs=100, batch_size=30, verbose=1, callbacks=[early_stop])

Epoch 1/100
228/228 [==============================] - 1s 4ms/step - loss: 0.2122
Epoch 2/100
228/228 [==============================] - 0s 319us/step - loss: 0.1379
Epoch 3/100
228/228 [==============================] - 0s 346us/step - loss: 0.0772
Epoch 4/100
228/228 [==============================] - 0s 324us/step - loss: 0.0343
Epoch 5/100
228/228 [==============================] - 0s 363us/step - loss: 0.0158
Epoch 6/100
228/228 [==============================] - 0s 328us/step - loss: 0.0176
Epoch 00006: early stopping
